### Pydantic とは

- python 3.6 から、「型アノテーション」という機能が追加されました。
- この 型アノテーション を利用して、型ヒントの提供や、型の確認や検証を行ってくれる機能などを提供してくれるのが [pydantic](https://pydantic-docs.helpmanual.io/) です。
- 今年の pycon でも pydantic について触れるプロポーザルが採用されていました。結構アツいライブラリみたいです。


#### python の型アノテーション
- 関数定義や変数作成時に、 `: 型` を追記
- editor によっては、コードを記述したタイミングで間違いを指摘してくれる

In [ ]:
name1 = "taro"
name2 : str = "taro"

In [ ]:
def add(a, b):
    return a + b

def add_type(a: int, b: int) -> int:
    return a + b


In [ ]:
add('1', '2')
# 実行する前から引数の型がおかしいことを指摘
add_type('1', '2')

### Pydantic 

1. `BaseModel` を継承して、pydantic モデルクラスを作成
    - `フィールド名: 型` で定義
    - 型は `typing` や pydanticが独自に定義している型を使う 
    - [typing Python 3.10.4 ドキュメント](https://docs.python.org/ja/3/library/typing.html)
    - [Field Types - pydantic](https://pydantic-docs.helpmanual.io/usage/types/#pydantic-types)
1. このクラスにデータを入れてオブジェクト化して使う

### FastApi で Pydantic を使うのは
1. request body を作る時
1. response データの表示方法を変えたい時


In [ ]:
%load_ext blackcellmagic

In [ ]:
# BaseModel を import
from pydantic import BaseModel


In [ ]:
from pydantic import HttpUrl

class Cat(BaseModel): # 継承
    id : int
    message: str 
    code: int 
    filepath: HttpUrl


In [ ]:
# データを入れて pydantic オブジェクトを作成
cat_1 = Cat(
    id=1,
    message="やったね",
    code=200,
    filepath="https://3.bp.blogspot.com/-IzBBa1iaxGc/XLQNJ_ysffI/AAAAAAABSbw/hgX31eDYY6QX5btrmZTNuMDm9JQL8B1ygCLcBGAs/s180-c/uchidenokoduchi_eto13_neko.png",
)

# 辞書渡しでもOK。
d = {
    "id": 2,
    "message": "てへぺろ",
    "code": 404,
    "filepath": "https://1.bp.blogspot.com/-d2MVqvUmxM0/V4SBCnW0-_I/AAAAAAAA8Qk/PZx69vFKAVgiAAOZzbeBWQC2erUmRdKoACLcB/s180-c/pet_tehe_cat.png",
}
cat_2 = Cat(**d)

In [ ]:
# 便利なメソッドがたくさん用意されている
print(dir(cat_1))

In [ ]:
# データへアクセス
cat_1.message

In [ ]:
cat_2.dict()

In [ ]:
cat_2.json()

In [ ]:
# 型チェック。フレンドリーな例外を返してくれる
d = {
    "id": 2,
    "message": "てへぺろ",
    "code": 404,
    "filepath": "file.png",
}
Cat(**d)

In [ ]:
# ValidationError クラスを使うと、例外を json で取得可
from pydantic import ValidationError
try:
    Cat(**d)
except ValidationError as e:
    print(e.json())

In [ ]:
# Pydanticモデルも型として使える

from typing import List 

class StatusCode(BaseModel):
    id : int 
    code: int 
    message: str 
    # cats フィールドには Cat 型のデータをリストで持つ定義とする。デフォルト値は空リスト
    cats: List[Cat] = [] 
    
    

In [ ]:
new_status_1 = {
    "id":1, 
    "code": 404,
    "message": "Not Found"
}

new_status_2 = {
    "id":2, 
    "code": 200,
    "message": "OK",
    "cats": [cat_1, cat_2]
}

status_1 = StatusCode(**new_status_1)
status_2 = StatusCode(**new_status_2)

In [ ]:
status_1.dict()

In [ ]:
status_2.dict()

### 既存のモデルを継承して使う

- ベースとなるモデルを作成しを継承する


In [ ]:

class BaseStatusCode(BaseModel):
    code: int 
    message: str 

class StatusCode(BaseStatusCode):
    id: int
    cats: List[Cat] = [] 



### `orm_mode = True`

- pydantic モデルへデータを流し込むには、通常モデルオブジェクトを作成するか、辞書で渡す。
- sqlalchemy 等の ORM データモデルのデータオブジェクトを扱うためには <font color=red>**必ず `orm_mode = True` 設定が必要**</font>

In [ ]:
# DataBase 

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, String, Integer, ForeignKey
from sqlalchemy.orm import relationship


DBFILE = "sqlite:///./SQLtest.db"
engine = create_engine(DBFILE, echo=True, connect_args={"check_same_thread": False})


SessionLocal = sessionmaker(
    bind=engine,
    autocommit=False,
    autoflush=False,
)

Base = declarative_base()


class TableStatusCode(Base):
    __tablename__ = "statuscodes"
    id = Column(Integer, primary_key=True, index=True)
    code = Column(Integer, unique=True)
    message = Column(String)

In [ ]:
# テストデータベースを作り、テストデータをInsertする
Base.metadata.create_all(bind=engine)
db = SessionLocal()
db.add(TableStatusCode(code=404, message="Not Found"))
db.commit()


In [ ]:
# データの確認
a_status = db.query(TableStatusCode).first()
a_status

In [ ]:
type(a_status)

In [ ]:
# orm_mode = True 設定無しでPydanticモデルを作成すると、データを読み込まない
class StatusCode(BaseModel):
    id : int 
    code: int 
    message: str 

# from_orm : テーブルデータオブジェクトを読み込むメソッド
print(StatusCode.from_orm(a_status))

In [ ]:
# orm_mode = True 設定をつけて、テーブルデータオブジェクトもPydanticで扱えるように設定する
class StatusCode(BaseModel):
    # id : int 
    code: int 
    message: str 

    class Config:
        orm_mode = True


print(StatusCode.from_orm(a_status))

In [ ]:
db.close()